In [ ]:
import pandas as pd
from pycaret.classification import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Cargar los datos
df = pd.read_csv("caso5_marketing_conversion.csv")

# 2. Análisis exploratorio: CTR por plataforma
df['ctr'] = df['conversion']
plt.figure(figsize=(8,4))
sns.barplot(data=df, x='plataforma', y='ctr')
plt.title("Tasa de conversión por plataforma")
plt.show()

# 3. Feature engineering: engagement combinado
df['engagement'] = np.sqrt(df['likes'] * 0.7 + df['comentarios'] * 1.5)

# Binning inteligente: duración en rangos
df['duracion_categoria'] = pd.cut(df['duracion_video_seg'],
                                  bins=[0, 30, 60, 120],
                                  labels=['Corto', 'Medio', 'Largo'])

# Interacción de plataforma con segmento
df['canal_segmento'] = df['plataforma'] + "_" + df['segmento']

# 4. PyCaret: setup con técnicas avanzadas
clf = setup(data=df,
            target='conversion',
            session_id=2025,
            categorical_features=['plataforma', 'segmento', 'duracion_categoria', 'canal_segmento'],
            ignore_features=['ctr', 'duracion_video_seg'],  # lo reemplazamos por la versión binned
            normalize=True,
            feature_selection=True,
            feature_interaction=True,
            remove_multicollinearity=True,
            transformation=True,
            transformation_method='yeo-johnson',
            silent=True)

# 5. Comparar modelos por F1 (conversion es evento raro)
best = compare_models(sort='F1')

# 6. Ajuste fino y visualización
final = tune_model(best, optimize='F1')
plot_model(final, plot='pr')  # Precision-Recall
plot_model(final, plot='confusion_matrix')

# 7. Interpretabilidad con SHAP
interpret_model(final)

# 8. Simular una campaña antes de lanzarla
nueva_campaña = pd.DataFrame({
    'plataforma': ['YouTube'],
    'segmento': ['Adulto'],
    'likes': [180],
    'comentarios': [45],
    'engagement': [np.sqrt(180 * 0.7 + 45 * 1.5)],
    'duracion_categoria': ['Medio'],
    'canal_segmento': ['YouTube_Adulto']
})
resultado = predict_model(final, data=nueva_campaña)
print("Conversión esperada:", resultado[['Label', 'Score']])

# 9. Exportar modelo para uso por el equipo creativo
save_model(final, 'modelo_marketing_conversion_predictor')
